# Sparse Autoencoder Training Demo

## Setup

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

from sparse_autoencoder import (
    SparseAutoencoder,
    TensorActivationStore,
    pipeline,
    create_src_dataloader,
)
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from sparse_autoencoder.src_data.datasets.neel_c4_tokenized import (
    collate_neel_c4_tokenized,
)
import torch
import wandb
import os

In [2]:
device = get_device()

/root/miniconda3/envs/arthurenv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


### Source Dataset

In [3]:
src_dataloader = create_src_dataloader(
    "NeelNanda/c4-code-tokenized-2b",
    collate_fn=collate_neel_c4_tokenized,
    shuffle_buffer_size=10_000,
    random_seed=0,
    batch_size=64,
)

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

### Source Model

In [4]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype=torch.float32)
src_d_mlp = src_model.cfg.d_mlp
src_d_mlp

(…)_1L512W_C4_Code/resolve/main/config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model_final.pth:   0%|          | 0.00/213M [00:00<?, ?B/s]

(…)igits/resolve/main/tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

(…)nizer-digits/resolve/main/tokenizer.json:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

(…)its/resolve/main/special_tokens_map.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Loaded pretrained model solu-1l into HookedTransformer


2048

### Activation Store

In [5]:
max_items = 2_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [6]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [8]:
wandb.init(
    project="sae",
    # entity="Arth",
    name = "Alan_Demo",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arthurconmy. Use `wandb login --relogin` to force relogin


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7ede269b10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f7e8c101bd0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7e8c101570, raw_cell="wandb.init(
    project="sae",
    # entity="Arth".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bvast_cold/root/sparse_autoencoder/demo.ipynb#X20sdnNjb2RlLXJlbW90ZQ%3D%3D> result=<wandb.sdk.wandb_run.Run object at 0x7f7e4fd29030>>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [9]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    src_dataloader=src_dataloader,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f7ede269b10>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f7e4fc98730, raw_cell="pipeline(
    src_model=src_model,
    src_model_a.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bvast_cold/root/sparse_autoencoder/demo.ipynb#X21sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Generate/Train Cycles: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Generate Activations:   0%|          | 0/1966080 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Train Autoencoder:   0%|          | 0/1966080 [00:00<?, ?it/s]